In [44]:
import pandas as pd
import numpy as np
import datetime

In [45]:
mc = pd.read_excel('data/match_1.xlsx')
mc.drop('№', axis=1, inplace=True)

In [46]:
mc

,index,Итог,гранада – атлетик,2\n0,вест хэм – ньюкасл юнайтед,0\n2,кристал пэлас – саутгемптон,1\n0,монпелье – ницца,3\n1,...,трабзонспор – бешикташ,1\n3,ростов – локомотив,0\n0,шеффилд юнайтед – вулверхэмптон,0\n2.1,алавес – бетис,0\n1,бордо – лион,0\n0.1
0,Андреев Иван,10,2:1,2,3:1,0,2:2,0,1:1,0,...,3:2,0,1:1,3,2:0,0,1:2,3,1:3,0
1,Афанасьев Владимир,11,1:1,0,2:0,0,1:2,0,2:0,3,...,2:1,0,2:0,0,0:2,5,1:2,3,1:2,0
2,Бакланов Сергей,6,1:0,2,1:1,0,0:2,0,2:2,0,...,3:1,0,1:2,0,2:3,2,1:1,0,1:2,0
3,Белькин Сергей,24,1:0,2,2:1,0,1:0,6,1:2,0,...,3:2,0,0:0,5,2:2,0,0:1,5,1:1,3
4,Гришин Антон,9,1:1,0,2:1,0,2:1,3,1:1,0,...,2:1,0,1:1,3,1:1,0,1:2,3,1:2,0
5,Дублицевич Александр,0,0:1,0,1:1,0,0:1,0,0:1,0,...,1:1,0,0:1,0,0:0,0,1:1,0,1:2,0
6,Еременко Владислав,10,0:2,0,1:2,2,0:2,0,1:2,0,...,2:2,0,2:1,0,0:2,5,1:2,3,1:3,0
7,Жигалов Александр,15,1:0,2,2:1,0,0:1,0,1:2,0,...,0:1,2,1:1,3,0:2,5,1:2,3,1:2,0
8,Зенин Максим,7,1:1,0,3:1,0,0:2,0,2:2,0,...,2:1,0,0:1,0,0:2,5,0:2,2,0:2,0
9,Зубатов Михаил,5,1:1,0,1:2,2,1:2,0,1:2,0,...,2:1,0,2:1,0,2:2,0,1:2,3,1:2,0


## Функции

In [47]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    
    print(str(player_name), match, 'Score:', true_score, 'Bet:', bet_score, 'Points:', points_granted)
    return

In [49]:
def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] = total
    print(bets.iloc[player_id, 0], total)

In [50]:
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].loc[match_ids] = 1
    return player_id

In [51]:
def get_match_ids(season=9, matchday=18):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['index', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

## Создание таблиц

In [59]:
sea = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
season_cols = ['Total Points', 'Game Points', 'Bomb Points', 'Bets', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

teams = pd.DataFrame(columns=['Team', 'Country', 'Matches', 'Wins', 'Draws', 'Losses', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques'] )
matches = pd.DataFrame(columns=['Hometeam', 'Awayteam', 'H_score', 'A_score', 'Season', 'Matchday', 'Date'] )

players = pd.DataFrame(mc['index'], columns=['index', 'bets']).rename(columns={'index': 'Name'})
season = pd.concat([pd.DataFrame(mc['index']), pd.DataFrame(np.zeros((mc['index'].shape[0],9)), columns=season_cols)], axis=1).rename(columns={'index':'Name'})

In [60]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                              'H_score': int(h_score), 'A_score': int(a_score),
                              'Season': 9, 'Matchday': 18, 'Date': datetime.datetime.now(), 'Bets': 0,
                              'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids()
bets = pd.DataFrame(pd.concat([mc['index'], pd.DataFrame(np.zeros((mc['index'].shape[0],8)), columns=sea)], axis=1), columns=bets_columns).rename(columns={'index': 'Name'})  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

<ipython-input-60-1cfb87966f38>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]


In [61]:
for player in range(0,players['Name'].shape[0]):
    for match in range(0,matches[matches['Season']==9][matches['Matchday']==18].index.shape[0]):
        proceed_bet(player,match)

H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Андреев Иван гранада - атлетик Score: 2:0 Bet: 2:1 Points: 2
Андреев Иван вест хэм - ньюкасл юнайтед Score: 0:2 Bet: 3:1 Points: 0
Андреев Иван кристал пэлас - саутгемптон Score: 1:0 Bet: 2:2 Points: 0
Андреев Иван монпелье - ницца Score: 3:1 Bet: 1:1 Points: 0
Андреев Иван цска - спартак москва Score: 3:1 Bet: 2:1 Points: 2
Андреев Иван трабзонспор - бешикташ Score: 1:3 Bet: 3:2 Points: 0
Андреев Иван ростов - локомотив Score: 0:0 Bet: 1:1 Points: 3
Андреев Иван шеффилд юнайтед - вулверхэмптон Score: 0:2 Bet: 2:0 Points: 0
Андреев Иван алавес - бетис Score: 0:1 Bet: 1:2 Points: 3
Андреев Иван бордо - лион Score: 0:0 Bet: 1:3 Points: 0
Афанасьев Владимир гранада - атлетик Score: 2:0 Bet: 1:1 Points: 0
Афанасьев Владимир вест хэм - ньюкасл юнайтед Score: 0:2 Bet: 2:0 Points: 0
Афанасьев Владимир кристал пэлас - саутгемптон Score: 1:0 Bet: 1:2 Points: 0
Афанасьев Владимир монпелье - ницца Score: 3:1 Bet: 2:0 Points: 3
Афанасьев Владимир цска - спартак москва Score: 3:1 Bet: 1:2 Points: 0

Муратов Игорь монпелье - ницца Score: 3:1 Bet: 0:1 Points: 0
Муратов Игорь цска - спартак москва Score: 3:1 Bet: 1:1 Points: 0
Муратов Игорь трабзонспор - бешикташ Score: 1:3 Bet: 1:0 Points: 0
Муратов Игорь ростов - локомотив Score: 0:0 Bet: 2:1 Points: 0
Муратов Игорь шеффилд юнайтед - вулверхэмптон Score: 0:2 Bet: 1:2 Points: 2
Муратов Игорь алавес - бетис Score: 0:1 Bet: 1:2 Points: 3
Муратов Игорь бордо - лион Score: 0:0 Bet: 1:2 Points: 0
Невский Леонид гранада - атлетик Score: 2:0 Bet: 1:1 Points: 0
Невский Леонид вест хэм - ньюкасл юнайтед Score: 0:2 Bet: 1:0 Points: 0
Невский Леонид кристал пэлас - саутгемптон Score: 1:0 Bet: 1:2 Points: 0
Невский Леонид монпелье - ницца Score: 3:1 Bet: 1:0 Points: 2
Невский Леонид цска - спартак москва Score: 3:1 Bet: 1:0 Points: 2
Невский Леонид трабзонспор - бешикташ Score: 1:3 Bet: 1:0 Points: 0
Невский Леонид ростов - локомотив Score: 0:0 Bet: 0:1 Points: 0
Невский Леонид шеффилд юнайтед - вулверхэмптон Score: 0:2 Bet: 1:2 Points: 2
Невск

In [62]:
for i in matches[matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Андреев Иван 10.0
Афанасьев Владимир 11.0
Бакланов Сергей 6.0
Белькин Сергей 24.0
Гришин Антон 9.0
Дублицевич Александр 0.0
Еременко Владислав 10.0
Жигалов Александр 15.0
Зенин Максим 7.0
Зубатов Михаил 5.0
Колодин Дмитрий 5.0
Конаков Никита 8.0
Котов Александр 5.0
Криеванс Владислав 6.0
Кузнецов Кирилл 9.0
Кучеренко Никита 2.0
Лесик Константин 5.0
Махмудов Руслан 7.0
Минеев Иван 13.0
Муратов Игорь 8.0
Невский Леонид 11.0
Новиков Александр 11.0
Оксанич Кирилл 7.0
﻿Сибиряков Георгий 10.0
Синичкин Александр 14.0
Титаренко Антон 8.0
Фёдоров Владимир 5.0
Холод Максим 12.0
Чапаров Александр 5.0
Шевчук Антон 16.0
Язынин Алексей 7.0


<ipython-input-50-693a0955bed7>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [63]:
season.sort_values(by=['Game Points'], ascending=False)

,Name,Total Points,Game Points,Bomb Points,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
3,Белькин Сергей,0.0,24.0,0.0,10.0,3.0,2.0,1.0,4.0,1.0
29,Шевчук Антон,0.0,16.0,0.0,10.0,1.0,2.0,2.0,5.0,1.0
7,Жигалов Александр,0.0,15.0,0.0,10.0,1.0,2.0,2.0,5.0,0.0
24,Синичкин Александр,0.0,14.0,0.0,10.0,1.0,1.0,3.0,5.0,0.0
18,Минеев Иван,0.0,13.0,0.0,10.0,0.0,3.0,2.0,5.0,0.0
27,Холод Максим,0.0,12.0,0.0,10.0,1.0,1.0,2.0,6.0,0.0
20,Невский Леонид,0.0,11.0,0.0,10.0,1.0,0.0,3.0,6.0,0.0
1,Афанасьев Владимир,0.0,11.0,0.0,10.0,1.0,2.0,0.0,7.0,0.0
21,Новиков Александр,0.0,11.0,0.0,10.0,0.0,3.0,1.0,6.0,0.0
23,﻿Сибиряков Георгий,0.0,10.0,0.0,10.0,0.0,2.0,2.0,6.0,0.0


## Bombs 

In [16]:
#bombs['Scorer'] = bombs['Бомбардир'].apply(lambda x: x.split(' (')[0])
#bombs['Team'] = bombs['Бомбардир'].apply(lambda x: x.split(' (')[1][:-1])
#bombs.drop('Бомбардир', axis=1, inplace=True)

#scorers['Name'] = bombs['Scorer']
#scorers['Team'] = bombs['Team']
#scorers.drop_duplicates('Name', inplace=True)
#scorers['Scored'] = 0
#bombs['Points'] = 0

KeyError: 'Бомбардир'

Суммируем очки

In [64]:
bombs = pd.read_excel('output/bombs_bets.xlsx', sheet_name="1", index_col=0)

In [65]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
1.0 Афанасьев Владимир
1.0 Бакланов Сергей
2.0 Белькин Сергей
1.0 Дублицевич Александр
1.0 Еременко Владислав
1.0 Жигалов Александр
1.0 Зенин Максим
1.0 Зубатов Михаил
1.0 Колодин Дмитрий
1.0 Кузнецов Кирилл
2.0 Муратов Игорь
2.0 Невский Леонид
3.0 Новиков Александр
1.0 ﻿Сибиряков Георгий
1.0 Титаренко Антон
2.0 Фёдоров Владимир
1.0 Шевчук Антон
1.0 Язынин Алексей


In [66]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [67]:
bets.sort_values(by=['Total Points'], ascending=False)

,Name,Total Points,Game Points,Bomb Points,Exact Score,Right Margin,Right Result,Wrong Result,Uniques,bet_hscore_0,...,points_6,bet_hscore_7,bet_ascore_7,points_7,bet_hscore_8,bet_ascore_8,points_8,bet_hscore_9,bet_ascore_9,points_9
3,Белькин Сергей,26.0,24.0,2.0,3.0,2.0,1.0,4.0,1.0,1,...,5.0,2,2,0.0,0,1,5.0,1,1,3.0
29,Шевчук Антон,17.0,16.0,1.0,1.0,2.0,2.0,5.0,1.0,2,...,3.0,0,0,0.0,2,0,0.0,1,1,3.0
7,Жигалов Александр,16.0,15.0,1.0,1.0,2.0,2.0,5.0,0.0,1,...,3.0,0,2,5.0,1,2,3.0,1,2,0.0
24,Синичкин Александр,14.0,14.0,0.0,1.0,1.0,3.0,5.0,0.0,0,...,5.0,1,2,2.0,1,0,0.0,0,1,0.0
21,Новиков Александр,14.0,11.0,3.0,0.0,3.0,1.0,6.0,0.0,0,...,0.0,1,3,3.0,1,1,0.0,0,1,0.0
18,Минеев Иван,13.0,13.0,0.0,0.0,3.0,2.0,5.0,0.0,2,...,0.0,1,3,3.0,1,2,3.0,0,2,0.0
20,Невский Леонид,13.0,11.0,2.0,1.0,0.0,3.0,6.0,0.0,1,...,0.0,1,2,2.0,0,1,5.0,1,2,0.0
27,Холод Максим,12.0,12.0,0.0,1.0,1.0,2.0,6.0,0.0,1,...,5.0,0,1,2.0,2,1,0.0,1,1,3.0
1,Афанасьев Владимир,12.0,11.0,1.0,1.0,2.0,0.0,7.0,0.0,1,...,0.0,0,2,5.0,1,2,3.0,1,2,0.0
6,Еременко Владислав,11.0,10.0,1.0,1.0,1.0,1.0,7.0,0.0,0,...,0.0,0,2,5.0,1,2,3.0,1,3,0.0


In [68]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('allinone.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
season.to_excel(writer, sheet_name='season_9')
matches.to_excel(writer, sheet_name='matches')
players.to_excel(writer, sheet_name='players')
teams.to_excel(writer, sheet_name='teams')
bets.to_excel(writer, sheet_name='bets_18')
bombs.to_excel(writer, sheet_name='bombs_bets_18')
#scorers.to_excel(writer, sheet_name='scorers_18')

writer.save()

In [69]:
matches

,Hometeam,Awayteam,H_score,A_score,Season,Matchday,Date,Bets,Exact Score,Right Margin,Right Result,Uniques,Wrong Result
0,гранада,атлетик,2,0,9,18,2020-09-25 08:30:47.204484,31.0,0.0,0.0,9.0,0.0,22.0
1,вест хэм,ньюкасл юнайтед,0,2,9,18,2020-09-25 08:30:47.210474,31.0,1.0,0.0,2.0,1.0,28.0
2,кристал пэлас,саутгемптон,1,0,9,18,2020-09-25 08:30:47.216453,31.0,1.0,4.0,0.0,1.0,26.0
3,монпелье,ницца,3,1,9,18,2020-09-25 08:30:47.221439,31.0,0.0,2.0,8.0,0.0,21.0
4,цска,спартак москва,3,1,9,18,2020-09-25 08:30:47.225428,31.0,0.0,2.0,6.0,0.0,23.0
5,трабзонспор,бешикташ,1,3,9,18,2020-09-25 08:30:47.230414,31.0,0.0,1.0,4.0,0.0,26.0
6,ростов,локомотив,0,0,9,18,2020-09-25 08:30:47.235437,31.0,5.0,11.0,0.0,0.0,15.0
7,шеффилд юнайтед,вулверхэмптон,0,2,9,18,2020-09-25 08:30:47.239417,31.0,5.0,2.0,12.0,0.0,12.0
8,алавес,бетис,0,1,9,18,2020-09-25 08:30:47.244407,31.0,2.0,10.0,3.0,0.0,16.0
9,бордо,лион,0,0,9,18,2020-09-25 08:30:47.248394,31.0,0.0,5.0,0.0,0.0,26.0


In [70]:
teams = pd.DataFrame(columns=['Team', 'Country', 'Matches', 'Wins', 'Draws', 'Losses', 'Bets', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques'] )

In [75]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,гранада,NaN,1,1,0,0,31.0,0.0,0.0,9.0,22.0,0.0
1,атлетик,NaN,1,0,0,1,31.0,0.0,0.0,9.0,22.0,0.0
2,вест хэм,NaN,1,0,0,1,31.0,1.0,0.0,2.0,28.0,1.0
3,ньюкасл юнайтед,NaN,1,1,0,0,31.0,1.0,0.0,2.0,28.0,1.0
4,кристал пэлас,NaN,1,1,0,0,31.0,1.0,4.0,0.0,26.0,1.0
5,саутгемптон,NaN,1,0,0,1,31.0,1.0,4.0,0.0,26.0,1.0
6,монпелье,NaN,1,1,0,0,31.0,0.0,2.0,8.0,21.0,0.0
7,ницца,NaN,1,0,0,1,31.0,0.0,2.0,8.0,21.0,0.0
8,цска,NaN,1,1,0,0,31.0,0.0,2.0,6.0,23.0,0.0
9,спартак москва,NaN,1,0,0,1,31.0,0.0,2.0,6.0,23.0,0.0


In [74]:
for match_id in matches[matches['Season']==9][matches['Matchday']==18].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [77]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('allinone.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
season.to_excel(writer, sheet_name='season_9')
matches.to_excel(writer, sheet_name='matches')
players.to_excel(writer, sheet_name='players')
teams.to_excel(writer, sheet_name='teams')
bets.to_excel(writer, sheet_name='bets_18')
bombs.to_excel(writer, sheet_name='bombs_bets_18')
#scorers.to_excel(writer, sheet_name='scorers_18')

writer.save()